In [0]:
with cte_data (a, y) as
(
select filter(transform(split(value, ''), (t,i) -> struct(i as x, t as v)), f -> f.v = '@').x
     , -1 + row_number() over W as y
--from read_files('/Volumes/waldar/fabien/aoc2025/aoc2025_04_input.txt', format => 'text', wholeText => false) as t (value)
  from values ('..@@.@@@@.')
            , ('@@@.@.@.@@')
            , ('@@@@@.@.@@')
            , ('@.@@@@..@.')
            , ('@@.@@@@.@@')
            , ('.@@@@@@@.@')
            , ('.@.@.@.@@@')
            , ('@.@@@.@@@@')
            , ('.@@@@@@@@.')
            , ('@.@.@@@.@.') as t (value)
window W as (order by monotonically_increasing_id() asc)
)
  ,  cte_grid (g) as
(
select flatten(array_agg(transform(a, x -> struct(x, y))))
  from cte_data
)
select aggregate( g
                , array()::array<struct<x:int,y:int,cnt:int>>
                , (acc, i) -> array_append(acc, struct( i.x
                                                      , i.y
                                                      , array_size(filter(g, f -> f.x in (i.x-1, i.x, i.x+1)
                                                                              and f.y in (i.y-1, i.y, i.y+1)
                                                                              and not (f.x = i.x and f.y = i.y))) as cnt
                                                      ))
                , acc -> array_size(filter(acc, p -> p.cnt < 4))
                ) as part1
     , aggregate( sequence(0,100)
                , struct(0 as r, g as arr)
                , (agg, x) -> aggregate( array(0)
                                       , array()::array<struct<x:int,y:int,cnt:int>>
                                       , (azz, z) -> transform( agg.arr
                                                              , i -> struct( i.x
                                                                           , i.y
                                                                           , array_size(filter(agg.arr, f -> f.x in (i.x-1, i.x, i.x+1)
                                                                                                         and f.y in (i.y-1, i.y, i.y+1)
                                                                                                         and not (f.x = i.x and f.y = i.y))) as cnt) )
                                       , azz -> struct( agg.r + array_size(filter(azz, p -> p.cnt < 4)) as r
                                                      , transform(filter(azz, p -> p.cnt >= 4), t -> struct(t.x, t.y)) as arr)
                                       )
                , agg -> agg.r
                ) as part2
  from cte_grid;




with cte_data (x, y) as
(
select filter(transform(split(value, ''), (t,i) -> struct(i as x, t as v)), f -> f.v = '@').x
     , -1 + dense_rank() over W as y
--from read_files('/Volumes/waldar/fabien/aoc2025/aoc2025_04_input.txt', format => 'text', wholeText => false) as t (value)
  from values ('..@@.@@@@.')
            , ('@@@.@.@.@@')
            , ('@@@@@.@.@@')
            , ('@.@@@@..@.')
            , ('@@.@@@@.@@')
            , ('.@@@@@@@.@')
            , ('.@.@.@.@@@')
            , ('@.@@@.@@@@')
            , ('.@@@@@@@@.')
            , ('@.@.@@@.@.') as t (value)
window W as (order by monotonically_increasing_id() asc)
)
  ,  cte_grid (x, y) as
(
select explode(x), y
  from cte_data
)
    select count(count(g2.x)) over W as part1
      from cte_grid as g1
 left join cte_grid as g2  on g2.x in (g1.x-1, g1.x, g1.x+1)
                          and g2.y in (g1.y-1, g1.y, g1.y+1)
                          and not(g2.x = g1.x and g2.y = g1.y)
  group by g1.x, g1.y
    having count(g2.x) < 4
    window W as ()
     limit 1;
